In [1]:
from time import perf_counter

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
model_path = "/opt/ml/final-project-level3-nlp-08/outputs/checkpoint-660"
eval_loss_path = "/opt/ml/final-project-level3-nlp-08/outputs/checkpoint-110"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

device = "cuda" if torch.cuda.is_available() else "cpu"
#tokenizer = AutoTokenizer.from_pretrained(eval_loss_path)
e_model = AutoModelForCausalLM.from_pretrained(eval_loss_path).to(device)

In [9]:
input_text = "경찰관이 말을 타고있다."
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Check generation time
t = perf_counter()

output = model.generate(
    input_ids,
    max_length=50,
    repetition_penalty=2.0,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    do_sample=True,
    top_k=30,
    top_p=0.95,
)
generated_text = tokenizer.decode(output[0])

spent = perf_counter() - t

print("generated text:", *map(lambda x: x.strip() + ".", generated_text.split(".")), sep="\n")
print(f"\ntime spent: {spent:.2f} sec")

generated text:
경찰관이 말을 타고있다.
그 사람은 나를 보고 말한다.
그는 내가 싫어.
나는 대답하지 않는다.
그저 조용히 걷기만 할뿐.
그리고 또 다시 나의 뒤통수에 대고 소리를 지른 다.
그러나 아무도 쳐다보지 못한다.
오직 나만이 그의 소리에 반응 한다.
왜.

time spent: 0.94 sec


In [10]:
t = perf_counter()

output = e_model.generate(
    input_ids,
    max_length=50,
    repetition_penalty=2.0,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    bos_token_id=tokenizer.bos_token_id,
    do_sample=True,
    top_k=30,
    top_p=0.95,
    bad_word_ids=[[30694],[44507],[31136, 31136]]
)
generated_text = tokenizer.decode(output[0])

spent = perf_counter() - t

print("generated text:", *map(lambda x: x.strip() + ".", generated_text.split(".")), sep="\n")
print(f"\ntime spent: {spent:.2f} sec")

generated text:
경찰관이 말을 타고있다.
그에게는 한 번도 본적없는 말.
하지만 그는 알고 있다.
그의 말에 따라 움직이고 있다는 것을.
.
시계가 그를 보고 있고 그가 손목을 긋고 있으며 그도 역시 똑같이 칼을 휘두르고 있지만 그들의 눈에는 모두 다르게.

time spent: 0.94 sec


In [102]:
tokenizer.encode(".")

[31136]